# 方策ベースの機械学習2

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [10]:
import sys

## pfrlパッケージのパスへの追加 

In [11]:
sys.path.append("/content/gdrive/MyDrive/repos/pfrl")

## インポート 

In [12]:
import pfrl
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

In [13]:
from tqdm.notebook import tqdm
import cv2

### 描画のための処理

[こちら](https://stackoverflow.com/a/61318224)あるいは[こちら](https://stackoverflow.com/a/61318224)を参考にした．

In [2]:
!apt-get install -y xvfb x11-utils
!pip install -q gym[box2d]==0.17.* pyvirtualdisplay==0.2.* PyOpenGL==3.1.* PyOpenGL-accelerate==3.1.*

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  libxxf86dga1
Suggested packages:
  mesa-utils
The following NEW packages will be installed:
  libxxf86dga1 x11-utils xvfb
0 upgraded, 3 newly installed, 0 to remove and 13 not upgraded.
Need to get 993 kB of archives.
After this operation, 2,981 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/main amd64 libxxf86dga1 amd64 2:1.1.4-1 [13.7 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/main amd64 x11-utils amd64 7.7+3build1 [196 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd64 xvfb amd64 2:1.19.6-1ubuntu4.8 [784 kB]
Fetched 993 kB in 1s (1,351 kB/s)
Selecting previously unselected package libxxf86dga1:amd64.
(Reading database ... 146374 files and directories currently installed.)
Preparing to unpack .../libxxf86dga1_2%3a1.1.4-1_amd64.deb ...
Unpacking libxxf86dga1:amd64 (2:

In [3]:
import pyvirtualdisplay
_display = pyvirtualdisplay.Display(visible=False,  # use False with Xvfb
                                    size=(1400, 900))
_ = _display.start()

## 離散行動・確率方策(リベンジ) 

In [5]:
import gym

前回のa2cによるペンデュラムの強化学習がうまくいかなかったので，今回はsoft-actor-criticで学習を行う．

In [6]:
concrete_env = gym.make("Pendulum-v0")
print("observation space:", concrete_env.observation_space)
print("action space:", concrete_env.action_space)

observation space: Box(-8.0, 8.0, (3,), float32)
action space: Box(-2.0, 2.0, (1,), float32)


### モデルの定義

モデルの構造はDDPGとほとんど一緒で，状態を入力として行動を出力する方策モデル(今回は確率を出力)と，状態と行動を入力として価値関数を出力する価値関数が必要である．

#### 方策モデルの定義 

In [14]:
class PolicyModel(nn.Module):
    def __init__(self, obs_size, action_dim, action_low, action_high):
        super().__init__()
        self.fc1 = nn.Linear(obs_size, 50)
        self.fc2 = nn.Linear(50, 50)
        self.fc3 = nn.Linear(50, action_dim)
        self.policy_bound = pfrl.nn.BoundByTanh(action_low, action_high)
        self.policy_head = pfrl.policies.GaussianHeadWithFixedCovariance(0.1)
        
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        
        #policy_x = F.relu(self.fc3(x))
        #policy_x = self.fc3(x)
        policy_x = 2 * torch.tanh(self.fc3(x))
        #policy_x =  self.policy_bound(self.fc3(x))
        out_policy = self.policy_head(policy_x)
        return out_policy

In [15]:
obs_size = concrete_env.observation_space.low.size
action_dim = concrete_env.action_space.low.size
action_low = concrete_env.action_space.low
action_high = concrete_env.action_space.high

policy_model = PolicyModel(obs_size, action_dim, action_low, action_high)

#### Q関数 

In [17]:
class QFunc(nn.Module):
    def __init__(self, obs_size, action_dim):
        super().__init__()
        self.concat_obs_action = pfrl.nn.ConcatObsAndAction()
        self.fc1 = nn.Linear(obs_size+action_dim, 50)
        self.fc2 = nn.Linear(50, 50)
        self.fc3 = nn.Linear(50, 1)
        
    def forward(self, x):
        x = self.concat_obs_action(x)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)  # 最後は活性化関数は必要は無い
        return x

In [18]:
q_func1 = QFunc(obs_size, action_dim)
q_func2 = QFunc(obs_size, action_dim)

#### マルチプロセス用のバッチ環境 

In [21]:
sys.path.append("/content/gdrive/MyDrive/rl_tutorial")
from tutorial_make_env_ver1 import make_concrete_env

In [22]:
process_number = 4
batch_concrete_env = pfrl.envs.MultiprocessVectorEnv([make_concrete_env for i in range(process_number)])

#### エージェントの定義 

ハイパーパラメータはこちらを参考とした([参考](https://github.com/pfnet/pfrl/blob/master/examples/mujoco/reproduction/soft_actor_critic/train_soft_actor_critic.py))．

In [23]:
policy_optimizer = torch.optim.Adam(policy_model.parameters(), lr=3e-4)
q_func1_optimizer = torch.optim.Adam(q_func1.parameters(), lr=3e-4)
q_func2_optimizer = torch.optim.Adam(q_func2.parameters(), lr=3e-4)

rbuf = pfrl.replay_buffers.ReplayBuffer(10 ** 6)

def burnin_action_func():
    """Select random actions until model is updated one or more times."""
    return np.random.uniform(concrete_env.action_space.low, concrete_env.action_space.high).astype(np.float32)

gpu = -1

gamma = 0.99 

replay_start_size =10000

#batch_size = 256
batch_size = 16  # 環境の数と異なることに注意

phi = lambda x: x.astype(np.float32, copy=False)

sac_agent = pfrl.agents.SoftActorCritic(
    policy_model,
    q_func1,
    q_func2,
    policy_optimizer,
    q_func1_optimizer,
    q_func2_optimizer,
    rbuf,
    phi=phi,
    gamma=gamma,
    replay_start_size=replay_start_size,
    gpu=gpu,
    minibatch_size=batch_size,
    burnin_action_func=burnin_action_func,
    entropy_target=-action_dim,
    temperature_optimizer_lr=3e-4,
)

### 学習のイテレーション

In [24]:
n_episodes = 1000  # エピソードの回数
max_episode_len = 200
for i in tqdm(range(1, n_episodes + 1)):
    
    obss = batch_concrete_env.reset()  # 観測のリセット
    R = np.zeros((process_number,))  # Return (sum ofrewards)
    t = 0  # time step
    while True:
        actions = sac_agent.batch_act(obss)
        obss, rewards, dones, _ = batch_concrete_env.step(actions)
        
        R += rewards
        t += 1
        
        resets = np.array([t == max_episode_len]*process_number)
        sac_agent.batch_observe(obss, rewards, dones, resets)
        if resets[0]:
            break
    
    if i%50 == 0:
        print("episode:{}, return:{}, t:{}".format(i, R, t))
    if i%100 == 0:
        print("statistics:", sac_agent.get_statistics())
print("Finshed")
batch_concrete_env.close()  # バッチで利用する場合は明示的にクローズ

episode:50, return:[-746.362587   -494.24369095 -754.00234931 -359.87311801], t:200
episode:100, return:[-642.7930444  -640.47807826 -637.48804755 -638.82893383], t:200
statistics: [('average_q1', -250.6291), ('average_q2', -250.6312), ('average_q_func1_loss', 175.21573196768762), ('average_q_func2_loss', 176.61977141141892), ('n_updates', 70001), ('average_entropy', -0.8836467), ('temperature', 5.087737736175768e-05)]
episode:150, return:[-196.29735236 -377.60357015 -367.33380597 -249.75897006], t:200
episode:200, return:[-119.67303694 -124.00614839   -1.24901911 -124.68486302], t:200
statistics: [('average_q1', -89.653534), ('average_q2', -89.59766), ('average_q_func1_loss', 12.165641034841537), ('average_q_func2_loss', 14.66298926949501), ('n_updates', 150001), ('average_entropy', -0.8836467), ('temperature', 1.2979199048857026e-08)]
episode:250, return:[-122.15057359 -121.97837793 -118.13973578 -221.19352129], t:200
episode:300, return:[-118.48758547 -119.4090003  -117.41768661 -12

#### 学習結果の可視化 

In [ ]:
fourcc = cv2.VideoWriter_fourcc('m', 'p', '4', 'v')
out = cv2.VideoWriter('movies/tutorial/pfrl_tutorial_sac_result.mp4', fourcc, 10, (500, 500))

max_episode_len = 200
with sac_agent.eval_mode():
    
    obs = concrete_env.reset()  # 観測のリセット
    R = 0  # Return (sum ofrewards)
    t = 0  # time step
    
    while True:
        action = sac_agent.act(obs)
        obs, reward, done, _ = concrete_env.step(action)
        R += reward
        t += 1
        reset = t == max_episode_len
        # 画像として取得，保存
        frame = concrete_env.render(mode="rgb_array")
        out.write(frame[:,:,::-1])       
        
        #sac_agent.observe(obs, reward, done, reset)
        if done or reset:
            break
            
concrete_env.close()
out.release()

print("episode length:", t)

episode length: 200


## 他のモデルもお試し 

### TRPO 

#### モデルの定義 

##### 方策モデルの定義

In [ ]:
class PolicyModel(nn.Module):
    def __init__(self, obs_size, action_dim, action_low, action_high):
        super().__init__()
        self.fc1 = nn.Linear(obs_size, 50)
        self.fc2 = nn.Linear(50, 50)
        self.fc3 = nn.Linear(50, action_dim)
        self.policy_bound = pfrl.nn.BoundByTanh(action_low, action_high)
        self.policy_head = pfrl.policies.GaussianHeadWithFixedCovariance(0.1)
        
    def forward(self, x):
        x = torch.tanh(self.fc1(x))
        x = torch.tanh(self.fc2(x))
        
        #policy_x = F.relu(self.fc3(x))
        #policy_x = self.fc3(x)
        policy_x = 2 * torch.tanh(self.fc3(x))
        #policy_x =  self.policy_bound(self.fc3(x))
        out_policy = self.policy_head(policy_x)
        return out_policy

In [ ]:
obs_size = concrete_env.observation_space.low.size
action_dim = concrete_env.action_space.low.size
action_low = concrete_env.action_space.low
action_high = concrete_env.action_space.high

policy_model = PolicyModel(obs_size, action_dim, action_low, action_high)

##### 価値関数の定義 

In [ ]:
class VFunc(nn.Module):
    def __init__(self, obs_size):
        super().__init__()
        self.fc1 = nn.Linear(obs_size, 50)
        self.fc2 = nn.Linear(50, 50)
        self.fc3 = nn.Linear(50, 1)
        
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)  # 最後は活性化関数は必要は無い
        return x

In [ ]:
v_func = VFunc(obs_size)

#### 重みの初期化

もとのexampleでは，直交行列による初期化が行われている．

In [ ]:
def ortho_init(layer, gain):
    nn.init.orthogonal_(layer.weight, gain=gain)
    nn.init.zeros_(layer.bias)
    
ortho_init(policy_model.fc1, gain=1)
ortho_init(policy_model.fc2, gain=1)
ortho_init(policy_model.fc3, gain=1e-2)
ortho_init(v_func.fc1, gain=1)
ortho_init(v_func.fc2, gain=1)
ortho_init(v_func.fc3, gain=1e-2)

#### 最適化手法

TRPOの最適化手法は方策モデルについては特殊な手法を用いるので，深層学習による最適化は価値関数に対してのみ行う．

In [ ]:
vf_opt = torch.optim.Adam(v_func.parameters())

#### エージェントの定義 

In [ ]:
obs_normalizer = pfrl.nn.EmpiricalNormalization(
    concrete_env.observation_space.low.size, clip_threshold=5
)

gpu = -1

trpo_update_interval = 5000

phi = lambda x: x.astype(np.float32, copy=False)


trpo_agent = pfrl.agents.TRPO(
    policy=policy_model,
    vf=v_func,
    vf_optimizer=vf_opt,
    obs_normalizer=obs_normalizer,
    gpu=gpu,
    update_interval=trpo_update_interval,
    phi=phi,
    max_kl=0.01,
    conjugate_gradient_max_iter=20,
    conjugate_gradient_damping=1e-1,
    gamma=0.995,
    lambd=0.97,
    vf_epochs=5,
    entropy_coef=0,
)

#### 学習のイテレーション 

In [ ]:
n_episodes = 1000  # エピソードの回数
max_episode_len = 200
for i in tqdm(range(1, n_episodes + 1)):
    obs = concrete_env.reset()  # 観測のリセット
    R = 0  # Return (sum ofrewards)
    t = 0  # time step
    while True:
        action = trpo_agent.act(obs)
        obs, reward, done, _ = concrete_env.step(action)
        R += reward
        t += 1
        reset = t == max_episode_len
        trpo_agent.observe(obs, reward, done, reset)
        if done or reset:
            break
    
    if i%50 == 0:
        print("episode:{}, return:{}".format(i, R))
    if i%100 == 0:
        print("statistics:", trpo_agent.get_statistics())
        
print("Finshed")

episode:50, return:-956.3156095101926
episode:100, return:-1601.5568378078367
statistics: [('average_value', -342.63205), ('average_entropy', -0.8836467), ('average_kl', 0.005610514956060797), ('average_policy_step_size', 0.75), ('explained_variance', 0.17451641137397245)]
episode:150, return:-1179.2167101503494
episode:200, return:-1070.0198208647985
statistics: [('average_value', -504.7983), ('average_entropy', -0.8836467), ('average_kl', 0.005198648635996506), ('average_policy_step_size', 0.75), ('explained_variance', 0.06952748522986674)]
episode:250, return:-1564.118827835543
episode:300, return:-1790.6384953417025
statistics: [('average_value', -577.014), ('average_entropy', -0.8836467), ('average_kl', 0.005378205969464034), ('average_policy_step_size', 0.75), ('explained_variance', 0.13961949396502193)]
episode:350, return:-1838.052834409432
episode:400, return:-887.4082104080612
statistics: [('average_value', -562.13275), ('average_entropy', -0.8836467), ('average_kl', 0.004578

### PPO 